# COVID-19 Vaccination and Deaths Analysis


In [ ]:
#Setup

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# connect to SQLite
conn = sqlite3.connect(":memory:")


In [ ]:
# Load the datasets
deaths = pd.read_csv("../data/covidDeaths.csv")
vaccinations = pd.read_csv("../data/covidVaccinations.csv")

# Quick look
deaths.head()
vaccinations.head()


In [ ]:
# Push dataframes into SQLite
deaths.to_sql("deaths", conn, if_exists="replace", index=False)
vaccinations.to_sql("vaccinations", conn, if_exists="replace", index=False)


#   SQL Queries

In [ ]:
#a. Select data by country/date

pd.read_sql_query("""
SELECT location, date, total_cases, new_cases, total_deaths, population
FROM deaths
WHERE continent IS NOT NULL
ORDER BY location, date;
""", conn)


In [ ]:
#b. Total cases vs deaths (likelihood of dying if infected)

pd.read_sql_query("""
SELECT location, date, total_cases, total_deaths, 
       (CAST(total_deaths AS FLOAT) / total_cases) * 100 AS DeathPercentage
FROM deaths
WHERE location = 'United Kingdom'
ORDER BY date;
""", conn)


In [ ]:
#c. Total cases vs population (percentage infected)

pd.read_sql_query("""
SELECT location, date, population, total_cases,
       (CAST(total_cases AS FLOAT) / population) * 100 AS PercentPopulationInfected
FROM deaths
WHERE location = 'United Kingdom'
ORDER BY date;
""", conn)


In [ ]:
#d. Countries with highest infection rate vs population

pd.read_sql_query("""
SELECT location, population, MAX(total_cases) AS HighestInfectionCount,
       MAX((CAST(total_cases AS FLOAT) / population)) * 100 AS PercentPopulationInfected
FROM deaths
WHERE continent IS NOT NULL
GROUP BY location, population
ORDER BY PercentPopulationInfected DESC;
""", conn)


In [ ]:
#e. Countries with highest death count

pd.read_sql_query("""
SELECT location, MAX(CAST(total_deaths AS INT)) AS TotalDeathCount
FROM deaths
WHERE continent IS NOT NULL
GROUP BY location
ORDER BY TotalDeathCount DESC;
""", conn)


In [ ]:
#f. Continents with highest death count

pd.read_sql_query("""
SELECT continent, MAX(CAST(total_deaths AS INT)) AS TotalDeathCount
FROM deaths
WHERE continent IS NOT NULL
GROUP BY continent
ORDER BY TotalDeathCount DESC;
""", conn)


In [ ]:
#g. Global numbers (aggregated)

pd.read_sql_query("""
SELECT date, SUM(new_cases) AS total_cases, SUM(new_deaths) AS total_deaths,
       (SUM(new_deaths) * 1.0 / NULLIF(SUM(new_cases),0)) * 100 AS DeathPercentage
FROM deaths
WHERE continent IS NOT NULL
GROUP BY date
ORDER BY date;
""", conn)


In [ ]:
#h. Join with vaccinations (rolling people vaccinated)

pd.read_sql_query("""
SELECT dea.continent, dea.location, dea.date, dea.population,
       vac.new_vaccinations,
       SUM(vac.new_vaccinations) OVER (
           PARTITION BY dea.location 
           ORDER BY dea.location, dea.date
       ) AS RollingPeopleVaccinated
FROM deaths dea
JOIN vaccinations vac
  ON dea.location = vac.location
  AND dea.date = vac.date
WHERE dea.continent IS NOT NULL
ORDER BY dea.location, dea.date;
""", conn)


In [ ]:
#6. Visualization Example

uk_data = pd.read_sql_query("""
    SELECT date, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred
    FROM vaccinations
    WHERE location = 'United Kingdom'
    ORDER BY date;
""", conn)

plt.figure(figsize=(12,6))
plt.plot(uk_data["date"], uk_data["people_vaccinated_per_hundred"], label="At least one dose")
plt.plot(uk_data["date"], uk_data["people_fully_vaccinated_per_hundred"], label="Fully vaccinated")
plt.title("UK Vaccination Coverage")
plt.xlabel("Date")
plt.ylabel("Percentage of Population")
plt.legend()
plt.grid(True)
plt.show()
